In [ ]:
import cv2 
import numpy as np 
from keras.applications import VGG16 
from keras.layers import Flatten, Dense, Input 
from keras.models import Model 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report 
import os 
# Function to convert RGB image to grayscale 
def rgb_to_gray(image): 
return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
# Function to apply MSR image enhancement 
def multi_scale_retinex(image, sigma_list): 
retinex = np.zeros_like(image, dtype=np.float32)  # Specify dtype=float32 explicitly 
epsilon = 1e-8  # Small constant to avoid division by zero 
for sigma in sigma_list: 
log_image = np.log(image.astype(np.float32) + epsilon) 
blurred_image = cv2.GaussianBlur(image, (0, 0), sigma).astype(np.float32) 
log_blurred_image = np.log(blurred_image + epsilon) 
retinex += log_image - log_blurred_image 
retinex = retinex / len(sigma_list) 
# Ensure retinex array is in the range [0, 255] and of type uint8 
retinex = np.clip(retinex, 0, 255).astype(np.uint8) 
return retinex 
# Function to perform weighted average fusion of images 
def weighted_average_fusion(image1, image2, alpha=0.5): 
return cv2.addWeighted(image1, alpha, image2, 1-alpha, 0) 
# Function to extract SIFT features 
def extract_sift_features(image): 
sift = cv2.SIFT_create() 
gray_image = rgb_to_gray(image) 
keypoints, descriptors = sift.detectAndCompute(gray_image, None) 
return descriptors 
# Load VGG16 model without classification layer 
vgg_model = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3))) 
for layer in vgg_model.layers: 
layer.trainable = False 
# Add classification layers on top of VGG16 
x = Flatten()(vgg_model.output) 
x = Dense(2048, activation='relu')(x) 
x = Dense(2, activation='softmax')(x) 
vgg_model = Model(inputs=vgg_model.input, outputs=x) 
# Compile the model 
vgg_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 
# Load CelebA-Spoof dataset (example) 
dataset_path = r"./spandy/train" 
real_images_folder = os.path.join(dataset_path, "REAL") 
spoof_images_folder = os.path.join(dataset_path, "FAKE") 
# Assuming you have spoof dataset with image paths and labels 
image_paths = [] 
labels = [] 
# Load real images 
for filename in os.listdir(real_images_folder): 
if filename.endswith(".jpg"): 
image_paths.append(os.path.join(real_images_folder, filename)) 
labels.append(0)  # Label 0 for real faces 
# Load spoof images 
for filename in os.listdir(spoof_images_folder): 
    if filename.endswith(".jpg"): 
        image_paths.append(os.path.join(spoof_images_folder, filename)) 
        labels.append(1)  # Label 1 for spoof faces 
# Read images, apply preprocessing, and create dataset 
X = [] 
y = [] 
for path, label in zip(image_paths, labels): 
    # Read image 
    image = cv2.imread(path) 
    # Apply MSR enhancement 
    enhanced_image = multi_scale_retinex(image, [15, 80, 250]) 
    # Resize images to fit VGG16 input size 
    image = cv2.resize(image, (224, 224)) 
    enhanced_image = cv2.resize(enhanced_image, (224, 224)) 
    # Perform weighted average fusion 
    fused_image = weighted_average_fusion(image, enhanced_image) 
    # Extract SIFT features 
    sift_features = extract_sift_features(fused_image) 
    # Append to dataset (concatenate features if needed) 
    if sift_features is not None and surf_features is not None: 
        features = np.concatenate((sift_features.flatten())) 
    elif sift_features is not None: 
        features = sift_features.flatten() 
    else: 
        features = np.array([]) 
    X.append(features) 
    y.append(label) 
X = np.array(X) 
y = np.array(y) 
# Ensure the dimensions of X are correct 
if X.ndim == 1: 
X = np.expand_dims(X, axis=0) 
print(f"Feature dataset shape: {X.shape}") 
print(f"Labels shape: {y.shape}") 